In [1]:
import pandas as pd
from scipy.stats import multinomial
import itertools
import random
import warnings
warnings.filterwarnings("ignore")

# Suppose you draw 10 coins at random from the bag (assume each of the five coin types will come up with equal probability for each draw). What is the probability that you will be able to buy an item for $1.00?

This experiment is a mutinomial experiment because:
1. consists of repeated trials.
2. each trial can result in a discrete number of outcomes.
3. on any trial, the probability that a particular coin will draw is constant.
4. trials (drawing coins) are independent (as we have a large bag), so  drawing  a particular coin on one trial doesn't affect the outcome of the other trials.

n=10 , k=5, p=1/5


In [2]:
def sums(k, n):
    if k == 1:
        yield (n,)
    else:
        for value in range(n + 1):
            for permutation in sums(k - 1, n - value):
                yield (value,) + permutation


k=5 #5 diffferent coin types
n=10 # number of trials 
L = list(sums(k,n))
print('total permutations of 5 integers between 0 and 10 (inclusive) with a summation of 10 is',len(L))

data=pd.DataFrame(columns=['p','n','d','q','h']) #pennies, nickels, dimes, quarters, and half dollars

j=0
for i in L[:]: 
    data.loc[j]= i
    j=j+1
    
data

total permutations of 5 integers between 0 and 10 (inclusive) with a summation of 10 is 1001


,p,n,d,q,h
0,0,0,0,0,10
1,0,0,0,1,9
2,0,0,0,2,8
3,0,0,0,3,7
4,0,0,0,4,6
...,...,...,...,...,...
996,9,0,0,0,1
997,9,0,0,1,0
998,9,0,1,0,0
999,9,1,0,0,0


In [3]:
#find permutations with total value greater equal $1
filtered_data=data.loc[data['p']*0.01+ data['n']*0.05+ data['d']*0.1+data['q']*0.25+ data['h']*0.5 >=1]

In [4]:
#compute multinomial probability for each permutation
rv = multinomial(10, [0.2, 0.2, 0.2, 0.2,0.2])
filtered_data['prob.']=filtered_data.apply(lambda x: rv.pmf(list(x)) , axis=1)
filtered_data

,p,n,d,q,h,prob.
0,0,0,0,0,10,1.024000e-07
1,0,0,0,1,9,1.024000e-06
2,0,0,0,2,8,4.608000e-06
3,0,0,0,3,7,1.228800e-05
4,0,0,0,4,6,2.150400e-05
...,...,...,...,...,...,...
967,7,0,0,1,2,3.686400e-05
968,7,0,0,2,1,3.686400e-05
970,7,0,1,0,2,3.686400e-05
976,7,1,0,0,2,3.686400e-05


In [5]:
print('The probability that you be will be able to buy an item for $1.00 is', filtered_data['prob.'].sum())

The probability that you be will be able to buy an item for $1.00 is 0.9347795968000017


# Suppose you draw 10 coins at random from the bag (assume each of the five coin types will come up with equal probability for each draw). What is the probability that you will be able to buy an item for $2.00?

In [6]:
#find permutations with total value greater equal $2
filtered_data2=data.loc[data['p']*0.01+ data['n']*0.05+ data['d']*0.1+data['q']*0.25+ data['h']*0.5 >=2]

#compute multinomial probability for each permutation
filtered_data2['prob.']=filtered_data2.apply(lambda x: rv.pmf(list(x)) , axis=1)
filtered_data2

,p,n,d,q,h,prob.
0,0,0,0,0,10,1.024000e-07
1,0,0,0,1,9,1.024000e-06
2,0,0,0,2,8,4.608000e-06
3,0,0,0,3,7,1.228800e-05
4,0,0,0,4,6,2.150400e-05
...,...,...,...,...,...,...
876,5,0,0,1,4,1.290240e-04
877,5,0,0,2,3,2.580480e-04
881,5,0,1,0,4,1.290240e-04
896,5,1,0,0,4,1.290240e-04


In [7]:
print('The probability that you be will be able to buy an item for $2.00 is', filtered_data2['prob.'].sum())

The probability that you be will be able to buy an item for $2.00 is 0.3670650880000005


# Now suppose that to pay for an item, you draw single coins one at a time from the bag (again, assume each of the five coin types will come up with equal probability for each draw) until you have enough for the item and then give those coins to the cashier. What is the expected amount of the change you get back from the cashier when the item costs $0.25?

My approach to solve this part was first to solve the exact problem by generating all possible trials. I noticed that this approach has time complexity as the value of the item increases. So i used simulation approach to find the approximate values.

##  Exact solution (inefficient approach)

only for small values of n (e.g. n=7) could find the solution!

In [8]:
%%time

results=[]
resultv=[]
prob=[]
coin_value = dict([('p', 1),
                  ('n',5),
                  ('d',10),
                  ('q',25),
                  ('h', 50) ])
def perm(n, seq):
    for j in itertools.product(seq, repeat=n):
        value=0
        coins=[]
        for i in range(n):#since worst case is to get all pennies
            value= value+coin_value[j[i]]
            coins.append(j[i])
            if value>= n: 
                break
        if coins not in results:
            results.append(coins)
            resultv.append(value)
            prob.append(1/5**len(coins))
    data=pd.DataFrame({'trial':results, 'value':resultv , 'prob' : prob} , columns =('trial', 'value', 'prob' ))    
    return data

n= 7 #value of the item in cents
data= perm(n, "pndqh")

data['sumprod']=(data['value']-n)* data['prob']
print('expected return to buy $0.07=', (data['sumprod'].sum())/100)
data

expected return to buy $0.07= 0.208457088
Wall time: 361 ms


,trial,value,prob,sumprod
0,"[p, p, p, p, p, p, p]",7,0.000013,0.000000
1,"[p, p, p, p, p, p, n]",11,0.000013,0.000051
2,"[p, p, p, p, p, p, d]",16,0.000013,0.000115
3,"[p, p, p, p, p, p, q]",31,0.000013,0.000307
4,"[p, p, p, p, p, p, h]",56,0.000013,0.000627
5,"[p, p, p, p, p, n]",10,0.000064,0.000192
6,"[p, p, p, p, p, d]",15,0.000064,0.000512
7,"[p, p, p, p, p, q]",30,0.000064,0.001472
8,"[p, p, p, p, p, h]",55,0.000064,0.003072
9,"[p, p, p, p, n]",9,0.000320,0.000640


## Simulation
used simulation to find approximate solutions

In [9]:
coin_value = dict([('p', 0.01),
                  ('n',0.05),
                  ('d',0.10),
                  ('q',0.25),
                  ('h', 0.50)])

def coin_sim(n,m):
    simulations=[]
    for i in range(m):
        sumofcoins = 0
        coins=[]
        while sumofcoins < n:
            coin = random.choices( population=['p','n','d','q','h'] , weights=[0.2, 0.2,0.2 ,0.2,0.2] , k=1)
            coins.append(coin[0])
            value=coin_value[coin[0]]
            sumofcoins = value+sumofcoins 
        simulations.append([coins] + [sumofcoins] + [sumofcoins-n] +[1/5**len(coins)] +[(sumofcoins-n)*1/5**len(coins)])
    df = pd.DataFrame(simulations,columns=['Trial','Coin_Value', 'Change','Probability' , 'sumprod'])
    return df


n=0.25 # to buy an item that costs $n 
m= 100000 # number of iterations for the simulation
df=coin_sim(n , m) 
print('expected amount of change when buy an item for $',n, 'is',df['Change'].mean() )
df

expected amount of change when buy an item for $ 0.25 is 0.16298360000001444


,Trial,Coin_Value,Change,Probability,sumprod
0,"[p, p, d, n, q]",0.42,0.17,0.00032,0.000054
1,"[p, h]",0.51,0.26,0.04000,0.010400
2,"[n, q]",0.30,0.05,0.04000,0.002000
3,[h],0.50,0.25,0.20000,0.050000
4,"[n, n, h]",0.60,0.35,0.00800,0.002800
...,...,...,...,...,...
99995,"[p, n, n, n, h]",0.66,0.41,0.00032,0.000131
99996,[h],0.50,0.25,0.20000,0.050000
99997,"[d, p, q]",0.36,0.11,0.00800,0.000880
99998,[h],0.50,0.25,0.20000,0.050000


# What is the standard deviation of the change you get back from the cashier when the item costs $0.25?

In [10]:
n=0.25 # to buy an item that costs $n 
m= 100000 # number of iterations for the simulation
df=coin_sim(n , m) 
print('the standard deviation of the change when the item costs $',n, 'is',df['Change'].std() )

the standard deviation of the change when the item costs $ 0.25 is 0.14184922044184295


# What is the expected amount of the change you get back from the cashier when the item costs $1.00?

In [12]:
n=1 # to buy an item that costs $n 
m= 100000 # number of iterations for the simulation
df=coin_sim(n , m) 
print('expected amount of change when buy an item for $',n, 'is',df['Change'].mean() )

expected amount of change when buy an item for $ 1 is 0.15686650000001348


# What is the standard deviation of the change you get back from the cashier when the item costs $1.00?

In [13]:
n=1 # to buy an item that costs $n 
m= 100000 # number of iterations for the simulation
df=coin_sim(n , m) 
print('the standard deviation of the change when the item costs $',n, 'is',df['Change'].std() )

the standard deviation of the change when the item costs $ 1 is 0.14025305883727135


# What is the expected amount of the change you get back from the cashier when the item costs $10.00?

In [14]:
n=10 # to buy an item that costs $n 
m= 100000 # number of iterations for the simulation
df=coin_sim(n , m) 
print('expected amount of change when buy an item for $',n, 'is',df['Change'].mean() )

expected amount of change when buy an item for $ 10 is 0.18230730000001585


# What is the standard deviation of the change you get back from the cashier when the item costs $10.00?

In [15]:
n=10 # to buy an item that costs $n 
m= 100000 # number of iterations for the simulation
df=coin_sim(n , m) 
print('the standard deviation of the change when the item costs $',n, 'is',df['Change'].std() )

the standard deviation of the change when the item costs $ 10 is 0.14159120970648573
